# QRT ENS Challenge Data 2023 - Benchmark

Ce notebook détaille la construction du benchmark de ce challenge - il peut également être utile aux participants pour se lancer dans la compétition. 

## Librairies

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

# Chargement des données

- `X_train` et `X_test` ont  $10$ colonnes qui représentent les même variables explicatives mais sur des périodes de temps différentes il y a au plus $10 604$ lignes. 

- `X_train` et `y_train` partagent la même colonne `DELIVERY_START` - chaque ligne a un DELIVERY_START unique associéz à une date et heure de livraison de l'électricité
. 

- La variable cible `spot_id_delta` de `y_train` correspond à l'écart entre le VWAP des transactions sur le marché infra-journalier (Intraday) et le prix SPOT pour 1MWh d'électricité (spot_id_delta = Intraday - SPOT) : si la valeur est positive, le prix Intraday est supérieur au prix SPOT et inversement.

- **On notera que certaines colonnes ont des valeurs manquantes**.


In [2]:
# After downloading the X_train/X_test/Y_train .csv files in your working directory:
X = pd.read_csv('../raw_data/X_train_Wwou3IE.csv')
y = pd.read_csv('../raw_data/y_train_jJtXgMX.csv')
X_rendu = pd.read_csv('../raw_data/X_test_GgyECq8.csv')

In [3]:
def to_seconds(hour_stamp):
    return int(hour_stamp.split(":")[0])*60*60

def date_stamp(df):
    df["year"] = df["DELIVERY_START"].apply(lambda x: x.split("-")[0])
    df["month"] = df["DELIVERY_START"].apply(lambda x: x.split("-")[1])
    df["day"] = df["DELIVERY_START"].apply(lambda x: (x.split("-")[2]).split(" ")[0])
    df["seconds"] = df["DELIVERY_START"].apply(lambda x: to_seconds(x.split(" ")[1]))
    return df
    
X = date_stamp(X)
X_rendu  = date_stamp(X_rendu)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

On manipule les données pour enlever les NaN qui ne nous intéressent pas: 

In [4]:
X_train_kept = X_train.dropna(subset=['coal_power_available', 'gas_power_available', 'nucelear_power_available', 'wind_power_forecasts_average', 'solar_power_forecasts_average', 'wind_power_forecasts_std', 'solar_power_forecasts_std'])
X_rendu_kept = X_train.dropna(subset=['coal_power_available', 'gas_power_available', 'nucelear_power_available', 'wind_power_forecasts_average', 'solar_power_forecasts_average', 'wind_power_forecasts_std', 'solar_power_forecasts_std'])

marked_train = pd.concat([X_train, X_train_kept]).drop_duplicates(keep=False)['DELIVERY_START']
marked_rendu = pd.concat([X_rendu, X_rendu_kept]).drop_duplicates(keep=False)['DELIVERY_START']

X_train = X_train[~X_train['DELIVERY_START'].isin(marked_train)]
y_train = y_train[~y_train['DELIVERY_START'].isin(marked_train)]
X_rendu = X_rendu[~X_rendu['DELIVERY_START'].isin(marked_train)]

In [5]:
y_train.to_csv('../data/y_train.csv')

PermissionError: [Errno 13] Permission denied: '../data/y_train.csv'

In [ ]:
X_train.head()

In [ ]:
X_train.describe()

In [ ]:
y_train.head()

In [ ]:
y_train.describe()

In [ ]:
X_test.head()

# Data analysis

Représentation graphique des données de test de notre modèle :

## 1) Histogramme des données d'apprentissage

In [ ]:
X_test.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

## 2) Répartition des *spot_id_delta* dans *y_train*

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(y_train['spot_id_delta'], color='g', bins=1000, hist_kws={'alpha': 0.4})

## 3) Corrélation générale entre les paramètres du modèle :

In [ ]:
df = pd.merge(X_train, y_train, on='DELIVERY_START', how='inner')
tab=df.corr()
fig, ax = plt.subplots()
im = ax.pcolor(tab, cmap='RdBu')
row_labels = tab.columns
col_labels = tab.index
ax.set_xticks(np.arange(tab.shape[1])+0.5, minor= False)
ax.set_yticks(np.arange(tab.shape[0])+0.5, minor= False)
ax.set_xticklabels(row_labels, minor = False)
ax.set_yticklabels(col_labels, minor = False)
plt.xticks(rotation=90)
fig.colorbar(im)
plt.show()

# Data cleaning

On s'intéresse ici à la présence de nombreuses valeurs NaN. Ces valeurs représentent un vrai manque pour l'entraînement de notre modèle car elles sont majoritairement situées sur les données très corrélées avec le *spot_id_delta*.

## 1) Valeurs aberrantes dans *y_train*

Comme vu lors de la partie Data Analysis, certaines des valeurs de *y_train* (3) ont des valeurs très éloignées des autres. On s'occupe donc ici de retirer ces valeurs qui ne semblent pas cohérentes pour notre modèle :

In [ ]:
threshold = 600

eliminated_starts = y_train[abs(y_train['spot_id_delta']) - threshold >= 0].DELIVERY_START

y_train = y_train[~y_train['DELIVERY_START'].isin(eliminated_starts)] # on ne sélectionne que les valeurs qui ne correspondent pas aux dates enlevées
X_train = X_train[~X_train['DELIVERY_START'].isin(eliminated_starts)] # de même ici pour être cohérent sur le nombre de lignes

print(len(y_train))
print(len(X_train))

In [ ]:
eliminated_starts.size

## 2) Prédiction de *load_forecast*

Il y a environ 1500 données manquantes, que l'on va prédire à l'aide d'un modèle de régression Ridge:

In [ ]:
sns.heatmap(X_train.isna(),cbar=False)

In [ ]:
sns.heatmap(X_rendu.isna(),cbar=False)

In [ ]:
predicted_spot_price_train = X_train['predicted_spot_price']
predicted_spot_price_rendu = X_rendu['predicted_spot_price']

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

y_load = X_train.dropna(subset=['load_forecast'])['load_forecast']
X_load = X_train.dropna(subset=['load_forecast'])
X_load = X_load.loc[:, X_load.columns != 'predicted_spot_price']
X_load = X_load.loc[:, X_load.columns != 'load_forecast']

X_train_load, X_test_load, y_train_load, y_test_load = train_test_split(X_load, y_load, test_size=0.3)

clf = LinearRegression()
clf.fit(X_train_load.set_index('DELIVERY_START'), y_train_load)
print('linear regression score: ', clf.score(X_test_load.set_index('DELIVERY_START'), y_test_load))

clf = GradientBoostingRegressor()
clf.fit(X_train_load.set_index('DELIVERY_START'), y_train_load)
print('gradient boosting regression score: ', clf.score(X_test_load.set_index('DELIVERY_START'), y_test_load))

clf = RandomForestRegressor()
clf.fit(X_train_load.set_index('DELIVERY_START'), y_train_load)
print('random forest regression score: ', clf.score(X_test_load.set_index('DELIVERY_START'), y_test_load))


On choisit donc ici la régression ayant le meilleur score : Random Forest. On peut maintenant prédire les valeurs manquantes pour le *load_forecast* :

In [ ]:
X_predict_load = X_train[X_train['load_forecast'].isna()]
predicted_spot_price1 = X_predict_load['predicted_spot_price']
load_forecast1 = X_predict_load['load_forecast']
X_predict_load = X_predict_load.loc[:, ~(X_predict_load.columns.isin(['predicted_spot_price', 'load_forecast']))]

X_predict_load.insert(1, 'load_forecast', clf.predict(X_predict_load.set_index('DELIVERY_START')))
X_predict_load.insert(9, 'predicted_spot_price', predicted_spot_price1)

#X_predict_load_rendu = X_rendu[X_rendu['load_forecast'].isna()]
#predicted_spot_price2 = X_predict_load_rendu['predicted_spot_price']
#load_forecast2 = X_predict_load_rendu['load_forecast']
#X_predict_load_rendu = X_predict_load_rendu.loc[:, ~(X_predict_load_rendu.columns.isin(['predicted_spot_price', 'load_forecast']))]

#X_predict_load_rendu.insert(1, 'load_forecast', clf.predict(X_predict_load_rendu.set_index('DELIVERY_START')))
#X_predict_load_rendu.insert(9, 'predicted_spot_price', predicted_spot_price2)

In [ ]:
X_train = pd.concat([X_train[~X_train['load_forecast'].isna()], X_predict_load])
#X_rendu = pd.concat([X_rendu[~X_rendu['load_forecast'].isna()], X_predict_load_rendu])
X_train.head()

In [ ]:
X_train['predicted_spot_price'] = predicted_spot_price_train
#X_rendu['predicted_spot_price'] = predicted_spot_price_rendu
X_train['predicted_spot_price'].describe()

## 3) Prédiction de *predicted_spot_price*

Maintenant, les valeurs pour de *load_forecast* NaN ont été remplacées. De part la forte corrélation en *predicted_spot_price* en *load_forecast*, et comme on dispose de toutes les valeurs de *load_forecast*, on peut maintenant créer un modèle capable de nous prédire les valeurs de *predicted_spot_price* manquantes:

In [ ]:
sns.heatmap(X_train.isna(),cbar=False) # vérification qu'il n'y a plus de valeur manquante dans load_forecast

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint

delivery_starts = X_train['DELIVERY_START']

y_spot = X_train.dropna(subset=['predicted_spot_price'])['predicted_spot_price']
X_spot = X_train.dropna(subset=['predicted_spot_price'])
X_spot = X_spot.loc[:, X_spot.columns != 'predicted_spot_price']
X_spot = X_spot.set_index('DELIVERY_START')

print('nombre de valeurs non NaN: ', len(y_spot))

X_train_spot, X_test_spot, y_train_spot, y_test_spot = train_test_split(X_spot, y_spot, test_size=0.25)

clf = LinearRegression()
clf.fit(X_train_spot, y_train_spot)
print('linear regression score: ', clf.score(X_test_spot, y_test_spot))

clf = RandomForestRegressor()
clf.fit(X_train_spot, y_train_spot)
print('random forest regression score: ', clf.score(X_test_spot, y_test_spot))

clf = GradientBoostingRegressor()
clf.fit(X_train_spot, y_train_spot)
print('gradient boosting regression score: ', clf.score(X_test_spot, y_test_spot))

In [ ]:
X_predict_spot = X_train[X_train['predicted_spot_price'].isna()]
X_predict_spot = X_predict_spot.loc[:, X_predict_spot.columns != 'predicted_spot_price']
X_predict_spot = X_predict_spot.set_index('DELIVERY_START')

X_predict_rendu = X_rendu[X_rendu['predicted_spot_price'].isna()]
X_predict_rendu = X_predict_rendu.loc[:, X_predict_rendu.columns != 'predicted_spot_price']
X_predict_rendu = X_predict_rendu.set_index('DELIVERY_START')

X_predict_spot.insert(8, 'predicted_spot_price', clf.predict(X_predict_spot))
X_predict_rendu.insert(8, 'predicted_spot_price', clf.predict(X_predict_rendu))
X_predict_spot.head()

In [ ]:
X_train = pd.concat([X_train[~X_train['predicted_spot_price'].isna()], X_predict_spot.reset_index('DELIVERY_START')])
X_rendu = pd.concat([X_rendu[~X_rendu['predicted_spot_price'].isna()], X_predict_rendu.reset_index('DELIVERY_START')])
sns.heatmap(X_train.isna(),cbar=False)

In [ ]:
sns.heatmap(X_rendu.isna(),cbar=False)

In [ ]:
y_train.head()

In [ ]:
X_train.to_csv('../data/X_train.csv')
X_rendu.to_csv('../data/X_test.csv')

On remarque bien que le database est maintenant dépourvu de valeurs NaN !

## Astuces et idées d'amélioration

- Réféchir à la modélation des différents facteurs qui font bouger les prix de l'électricité dans chaque pays pourra être utile. 

- Le jeu de données est relativement petit - c'est un "small data challenge" - alors attention à ne pas surapprendre les paramètres de vos modèles ! Il sera certainement utile de mettre en place de bonnes pratiques de validation croisée.


In [ ]:
lm=LinearRegression()
Z = X_train_clean
lm.fit(Z, Y_train['spot_id_delta'])
Y_hat = lm.predict(X_train_clean)
sns.residplot(Z['load_forecast'], Y_train['spot_id_delta'])

In [ ]:
Input = [('Scale',StandardScaler()),('polynomial',PolynomialFeatures(degree=2)),('model',LinearRegression())]
pipe=Pipeline(Input)
pipe.fit(X_train_clean,Y_train['spot_id_delta'])
Y_hat=pipe.predict(X_train_clean)

On voit quand on trace les résidus de la regression linéaire entre le load_forecast en France et la différence, une distribution équiprobable partout ce qui laisse penser que la regression linéaire est correcte


In [ ]:
RSQ_test= []
lr=LinearRegression()
X_test_clean = X_test.drop(['DELIVERY_START'], axis=1).fillna(0)
Y_test_clean = Y_train.drop(['DELIVERY_START'], axis=1).fillna(0)
order = [1,2,3,4,5]
for n in order :
    pr = PolynomialFeatures(degree=n)
    x_train_pr=pr.fit_transform(X_train_clean[['load_forecast']])
    x_test_pr=pr.fit_transform(X_test_clean[['load_forecast']])
    lr.fit(x_train_pr,Y_train_clean)
    RSQ_test.append(lr.score(x_test_pr,Y_test_clean))

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

RidgeModel = Ridge(alpha=0.1)
RidgeModel.fit(X_train_clean,Y_train_clean)
Yhat=RidgeModel.predict(X_train_clean)

parameters = [{'alpha': [0.001,2,10,100,1000,10000,100000,1000000000]}]
RR=Ridge()
Grid1=GridSearchCV(RR,parameters,cv=5)
Grid1.fit(X_train_clean,Y_train_clean)
Grid1.best_estimator_
scores=Grid1.cv_results_


for param, mean_val in zip(scores['params'],scores['mean_test_score']):
    print(param, "R^2 on test data:", mean_val)
    
